In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import gym
import numpy
import onnx
import onnxruntime
from gym import wrappers
import os
from typing import Any, Dict, List, Optional
import glob

import argparse
import os
import time
#import caffe2.python.onnx.backend as onnx_caffe2

# import geffnet
# from base.rl.ppo import PPO

print(torch.__version__)
print(onnx.__version__)
print(onnxruntime.__version__)

1.2.0
1.6.0
1.6.0


In [3]:
class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)

class Global_Actor(nn.Module):
    def __init__(self):
        super().__init__()
        self.actor = nn.Sequential(  # (8, G, G)
            nn.Conv2d(8, 8, 3, padding=1),  # (8, G, G)
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 4, 3, padding=1),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 4, 5, padding=2),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 2, 5, padding=2),  # (2, G, G)
            nn.BatchNorm2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 1, 5, padding=2),  # (1, G, G)
            Flatten(),  # (G*G, )
            nn.Sigmoid(),  # added for non-negative
        )
        
    def forward(self, inputs):
        x2 = self.actor(inputs)
        return x2

In [29]:
net = Global_Actor().actor
# print(list(net.parameters()))
# print(net.state_dict())

In [30]:
print(net)
# print(list(net.parameters()))
torch.save(net.state_dict(), "pth/net_params.pkl")
torch.save(net.state_dict(), "pth/net_params.pth")

Sequential(
  (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv2d(4, 4, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (7): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Conv2d(4, 2, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (10): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Conv2d(2, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (13): Flatten()
  (14): Sigmoid()
)


In [31]:
c = Global_Actor()

In [33]:
c.actor.load_state_dict(torch.load("pth/net_params.pth"))

<All keys matched successfully>